# Q2) Predicting Food Delivery Time
In this question we are aksed to implement linear regression and gradient descent by working on a real-world dataset on food delivery time.

In [6]:
# Imorting the necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('Food_Delivery_Times.csv')
print(df.isnull().sum())

Order_ID                   0
Distance_km                0
Weather                   30
Traffic_Level             30
Time_of_Day               30
Vehicle_Type               0
Preparation_Time_min       0
Courier_Experience_yrs    30
Delivery_Time_min          0
dtype: int64


Now we can see that the fields 'Weather', 'Traffic_Level', 'Time_of_Day' and 'Courier_Experience_yrs' are have null entires. Let us handle the missining values for each of these as follows:
1) If the field is a categorical field, we fill the misssing entries with the mode (most occuring) of the field
2) If the field is a numeric field, we fill the missing entries with the mean of the field

In [7]:
categorical_fields = ['Weather', 'Traffic_Level', 'Time_of_Day']
for column in categorical_fields:
    try:
        df[column].fillna(df[column].mode()[0], inplace=True)
    except Exception as e:
        print('There seems to be an error:',e)

df['Courier_Experience_yrs'].fillna(df['Courier_Experience_yrs'].mean(), inplace=True)

# Verifying if there are any missing values
print(df.isnull().sum())

Order_ID                  0
Distance_km               0
Weather                   0
Traffic_Level             0
Time_of_Day               0
Vehicle_Type              0
Preparation_Time_min      0
Courier_Experience_yrs    0
Delivery_Time_min         0
dtype: int64


/tmp/ipykernel_29686/3320626411.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
/tmp/ipykernel_29686/3320626411.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

Now we are asked to encode the necessary fields. Encoding refers to the process of converting categorical (non-numeric) values into numeric values so that their usage becomes easier in methods like linear regression. For the categorical columns 'Weather' and 'Time_of_Day' we go with the One-Hot Encoding and for 'Traffic_Level' we go with Ordinal Encoding

In [11]:
# One-Hot encoding for the categorical fields 'Weather' and 'Time_of_Day'
weather_ohe = pd.get_dummies(file['Weather'], prefix='Weather')
df = pd.concat([file, weather_ohe], axis=1)
df.drop(columns=['Weather'], inplace=True)

time_of_day_ohe = pd.get_dummies(df['Time_of_Day'], prefix='Time_of_Day')
df = pd.concat([df, time_of_day_ohe], axis=1)
df.drop(columns=['Time_of_Day'], inplace=True)

# Ordinal encoding for Traffic_Level
traffic_level_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
df['Traffic_Level'] = df['Traffic_Level'].map(traffic_level_mapping)

print(df.head())

   Order_ID  Distance_km  Traffic_Level Vehicle_Type  Preparation_Time_min  \
0       522         7.93            1.0      Scooter                    12   
1       738        16.42            2.0         Bike                    20   
2       741         9.52            1.0      Scooter                    28   
3       661         7.44            2.0      Scooter                     5   
4       412        19.03            1.0         Bike                    16   

   Courier_Experience_yrs  Delivery_Time_min  Weather_Clear  Weather_Foggy  \
0                     1.0                 43          False          False   
1                     2.0                 84           True          False   
2                     1.0                 59          False           True   
3                     1.0                 37          False          False   
4                     5.0                 68           True          False   

   Weather_Rainy  Weather_Snowy  Weather_Windy  Time_of_Day_Af